## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from numpy import linalg as LA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
# прочитаем файл
data = pd.read_csv('/datasets/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
# проверим пропуски
data.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [4]:
# проверим дубликаты
data.duplicated().sum()

153

In [5]:
# удалим дубликаты
data = data.drop_duplicates().reset_index(drop=True)

In [6]:
data.duplicated().sum()

0

In [7]:
# посмотрим общую информацию
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                4847 non-null   int64  
 1   Возраст            4847 non-null   float64
 2   Зарплата           4847 non-null   float64
 3   Члены семьи        4847 non-null   int64  
 4   Страховые выплаты  4847 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 189.5 KB


In [8]:
# приведем данные к нужным типам
data['Возраст'] = data['Возраст'].astype('int')
data['Зарплата'] = data['Зарплата'].astype('int')
data[['Возраст', 'Зарплата']].tail()

,Возраст,Зарплата
4842,28,35700
4843,34,52400
4844,20,33900
4845,22,32700
4846,28,40600


In [9]:
# проверим аномалии
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,4847.000000,4847.000000,4847.000000,4847.000000,4847.000000
mean,0.498453,31.023932,39895.811223,1.203425,0.152259
std,0.500049,8.487995,9972.952441,1.098664,0.468934
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33200.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Данные корректны, аномалий нет, пропусков нет. Были обнаружены и удалены дубликаты.

## Умножение матриц

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** b) Не изменится.

**Обоснование:** 

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y_1, y_2$ — векторы целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w_1, w_2$ — векторы весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a_1 = Xw_1$$
$$
a_2 = XPw_2
$$


Формула вычисления весов:

$$
w_1 = (X^T X)^{-1} X^T y_1$$
$$
w_2 = ((XP)^T (XP))^{-1} (XP)^T y_2$$


Из теории нам известно:

$$
AE = EA = A$$
$$
AA^{-1} = A^{-1}A = E$$
$$ 
(AB)^{-1} = B^{-1}A^{-1}$$
$$
(AB)^T = B^TA^T$$


Необходимо доказать, что предсказания не изменятся (равны):

$$
a_1 = a_2$$
$$
Xw_1 = XPw_2$$


Пользуясь теорией, произведем преобразования:
$$
w_2 = ((XP)^T (XP))^{-1} (XP)^T y_2 =$$
$$
(P^TX^TXP)^{-1}P^TX^Ty_2 =$$
$$
P^{-1}(P^TX^TX)^{-1}P^TX^Ty_2 =$$
$$
P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty_2 =$$
$$
P^{-1}(X^TX)^{-1}EX^Ty_2 =$$
$$
P^{-1}(X^TX)^{-1}X^Ty_2$$


Произведем доказательство:

$$
X(X^T X)^{-1} X^T y_1 = XPP^{-1}(X^TX)^{-1}X^Ty_2$$
$$
X(X^T X)^{-1} X^T y_1 = XE(X^TX)^{-1}X^Ty_2$$
$$
X(X^T X)^{-1} X^T y_1 = X(X^TX)^{-1}X^Ty_2$$


Получается, что $a_1 = a_2$, а это значит, что качество модели не изменится.

## Алгоритм преобразования

**Алгоритм**

1. Отделить матрицу признаков и вектор целевого признака.
2. Так как обратная матрица может быть найдена только для квадратной матрицы, нужно создать случайную обратимую квадратную матрицу со стороной, равной колучеству признаков.
3. В отдельную переменную сохранить обратную матрицу от случайной квадратной матрицы, чтобы иметь возможность восстановить исходную матрицу признаков.
4. Создать собственный класс модели, внутри которого происходит умножение матрицы признаков на случайную обратимую матрицу.

**Обоснование**

При вычислении предсказания, случайная матрица как бы "съедается", превращаясь в единичную. Таким образом формула вычесления весов становится такой, как была до преобразования.

## Проверка алгоритма

### Создание необходимых переменных

In [10]:
# разобьем на признаки и целевой признак
features = data.drop(['Страховые выплаты'], axis=1)
target = data['Страховые выплаты']

In [11]:
# разделим на обучающую и тестовую выборки
features_train, features_test, target_train, target_test = (
    train_test_split(features, target, test_size=0.25, random_state=5))

In [12]:
# создадим случаную обратимую матрицу
random_matrix = np.random.normal(size=(features.shape[1], features.shape[1]))
random_matrix

array([[-2.33908378, -0.21287071,  0.36323943, -1.86473796],
       [-0.99206804, -0.24075521,  1.15621615, -0.82664708],
       [ 0.34534724, -0.83526248,  1.55435592, -0.3662362 ],
       [ 0.33337049,  0.24623838, -0.12758269, -0.25483428]])

In [13]:
# сохраним в отдельную переменную обратную от random_matrix
decrypt_matrix = LA.inv(random_matrix)
decrypt_matrix

array([[-0.01476484, -0.47675392,  0.44178067,  1.01965759],
       [-0.82581566,  1.97874102, -1.17156418,  1.30782466],
       [-0.56625583,  1.31738786, -0.19224602,  0.14641427],
       [-0.53377917,  0.62876317, -0.45786682, -1.3998099 ]])

### Проверка восстановления исходных данных

In [14]:
# посмотрим, как выглядит исходная матрица признаков
features_train.head()

,Пол,Возраст,Зарплата,Члены семьи
4494,0,35,55000,1
1911,1,27,44900,1
3646,0,29,42900,4
2577,0,30,41300,1
3148,1,41,38700,2


In [15]:
# посмотрим, как выглядит матрица признаков, умноженная на случайную матрицу
A = features_train@random_matrix
A.head()

,0,1,2,3
4494,18959.708926,-45947.616840,85529.915503,-20172.178614
1911,15477.299311,-37509.752576,69822.034235,-16468.444531
3646,14787.959900,-35838.757532,66714.888843,-15736.525186
2577,14233.412144,-34503.317027,64229.458338,-15150.609406
3148,13322.590870,-32334.249507,60201.086984,-14209.607970


In [16]:
# умножим матрицу A на дешифрующую матрицу 
B = A@decrypt_matrix
B = B.round().astype('int')
B.head()

,0,1,2,3
4494,0,35,55000,1
1911,1,27,44900,1
3646,0,29,42900,4
2577,0,30,41300,1
3148,1,41,38700,2


Как видим, матрица B совпадает с исходной матрицей признаков.

### Создание класса модели

In [17]:
class My_LinearRegression:
    def fit(self, features, target, rnd_matrix):
        features = features @ rnd_matrix
        X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)
        y = target
        w = LA.inv(X.T.dot(X)).dot(X.T).dot(y) 
        self.w = w[1:]
        self.w0 = w[0]
 
    def predict(self, test, rnd_matrix):
        return test.dot(rnd_matrix).dot(self.w) + self.w0

In [18]:
model_my_lr = My_LinearRegression()
model_my_lr.fit(features_train, target_train, random_matrix)
predictions_my_lr = model_my_lr.predict(features_test, random_matrix)
print('Метрика R2 по зашифрованным данным', r2_score(target_test, predictions_my_lr))

Метрика R2 по зашифрованным данным 0.4361718726663325


In [19]:
model_lr = LinearRegression()
model_lr.fit(features_train, target_train)
predictions_lr = model_lr.predict(features_test)
print('Метрика R2 по исходным данным', r2_score(target_test, predictions_lr))

Метрика R2 по исходным данным 0.4361718739985623


##  Вывод

Метрики, полученные по зашифрованным и исходным данным почти идентичные, разняться лишь на 7-ом знаке после запятой, а значит качество модели не меняется. Матричное умножение можно использовать как рабочий метод для шифрования данных, а за счет того, что матрица шифрования генерируется случайным образом, восстановить исходные данные будет сложно. 